In [1]:
import urllib
import pandas as pd
import sys
import collections
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from time import time
from sklearn.metrics import recall_score,precision_score,accuracy_score

judges = ['AScalia', 'AMKennedy', 'CThomas','RBGinsburg', 'SGBreyer', 'JGRoberts', 'SAAlito', 'SSotomayor','EKagan']
train_cols = ['issue','issueArea','lcDispositionDirection','petitioner','respondent']



/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "


In [2]:
def read():
    facts = pd.read_table('cases_55-15.csv',sep='|')
    df_justice_all = pd.read_csv('SCDB_2015_02_justiceCentered_LegalProvision.csv', encoding="ISO-8859-1")
    return df_justice_all, facts

In [3]:
def tfvecto(list_of_facts, x=1):
    tfvec = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'), ngram_range=(1, x))
    X = tfvec.fit_transform(list_of_facts)
    return X

In [7]:
    df_justice_all, facts = read()
    df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')
    cur_df = df[df['justiceName'].isin(judges)] #DF with only the current judges + Scalia
    cur_df = cur_df[cur_df.direction.notnull()] #DF where each Judge has a direction.
                                                # Can't train otherwise
    cur_df.reset_index(inplace=True)

In [8]:
X = tfvecto(cur_df.facts_of_the_case, 2)

In [18]:
X

<11951x14658 sparse matrix of type '<class 'numpy.float64'>'
	with 892082 stored elements in Compressed Sparse Row format>

In [10]:
jn = pd.get_dummies(cur_df.justiceName, sparse=True)

In [15]:
from scipy import sparse

In [16]:
A = sparse.coo_matrix(jn)

In [19]:
Q = sparse.hstack([X, A])

In [29]:
W = Q.tocsr()

In [32]:
cross_v(W, cur_df)

Fitting model...
Score:  0.762442492681
Baseline:  0.548724383103
Time: 471.291s
---------------
Fitting model...
Score:  0.782845188285
Baseline:  0.554811715481
Time: 418.623s
---------------
Fitting model...
Score:  0.792887029289
Baseline:  0.538912133891
Time: 450.694s
---------------
Fitting model...
Score:  0.784518828452
Baseline:  0.540167364017
Time: 411.346s
---------------
Fitting model...
Score:  0.783682008368
Baseline:  0.538075313808
Time: 191.447s
---------------
Score of CLF =  0.781275109415
Baseline of =  0.54413818206


,predicted,direction,justiceName,issueArea
0,2,2.0,AScalia,2.0
1,2,2.0,AScalia,2.0
2,2,2.0,AMKennedy,2.0
3,2,2.0,AMKennedy,2.0
4,2,1.0,CThomas,2.0
5,2,1.0,CThomas,2.0
6,2,2.0,RBGinsburg,2.0
7,2,2.0,RBGinsburg,2.0
8,2,2.0,SGBreyer,2.0
9,2,2.0,SGBreyer,2.0


In [33]:
cross_v(W, cur_df)

Fitting model...
Score:  0.771643663739
Baseline:  0.544123797574
Time: 204.097s
---------------
Fitting model...
Score:  0.771129707113
Baseline:  0.54769874477
Time: 197.915s
---------------
Fitting model...
Score:  0.767782426778
Baseline:  0.548953974895
Time: 228.456s
---------------
Fitting model...
Score:  0.775732217573
Baseline:  0.542677824268
Time: 226.543s
---------------
Fitting model...
Score:  0.794979079498
Baseline:  0.537238493724
Time: 230.847s
---------------
Score of CLF =  0.77625341894
Baseline of =  0.544138567046


,predicted,direction,justiceName,issueArea
0,2,2.0,AScalia,2.0
1,2,2.0,AScalia,2.0
2,2,2.0,AMKennedy,2.0
3,2,2.0,AMKennedy,2.0
4,2,1.0,CThomas,2.0
5,2,1.0,CThomas,2.0
6,2,2.0,RBGinsburg,2.0
7,2,2.0,RBGinsburg,2.0
8,2,2.0,SGBreyer,2.0
9,2,2.0,SGBreyer,2.0


In [ ]:
    X_array = X.toarray()
    facts_df = pd.DataFrame(X_array)

    df = pd.concat([cur_df[train_cols+['direction', 'justiceName']], facts_df], axis=1)
    # print ('Len with nulls: ',len(df))
    df = df[df.notnull().all(axis=1)].reset_index()
    # print ('Len with no Nulls: ', len(df))
    # df = df[df.notnull().all(axis=1)==False].reset_index()
    x = df.ix[:,df.columns != 'direction']
    # x = df.ix[:, [col for col in df.columns if col not in train_cols+['direction']]]
    # x = df.ix[:, train_cols]
    
    # x = x.ix[:, x.columns != 'justiceName']
    
    x = pd.get_dummies(x)

In [4]:
def run(n=0, bigram=False):
    df_justice_all, facts = read()
    df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')
    cur_df = df[df['justiceName'].isin(judges)] #DF with only the current judges + Scalia
    cur_df = cur_df[cur_df.direction.notnull()] #DF where each Judge has a direction.
                                                # Can't train otherwise
    cur_df.reset_index(inplace=True)
    #bigram=False
    if n>1:
        bigram = True
    if bigram:
        X = tfvecto(cur_df.facts_of_the_case, 2)
    else:
        X = tfvecto(cur_df.facts_of_the_case)

    if (n==1 or n==2):
        cross_v(X, cur_df)
        return

    # Facts + variables case

    X_array = X.toarray()
    facts_df = pd.DataFrame(X_array)

    df = pd.concat([cur_df[train_cols+['direction', 'justiceName']], facts_df], axis=1)
    # print ('Len with nulls: ',len(df))
    df = df[df.notnull().all(axis=1)].reset_index()
    # print ('Len with no Nulls: ', len(df))
    # df = df[df.notnull().all(axis=1)==False].reset_index()
    x = df.ix[:,df.columns != 'direction']
    # x = df.ix[:, [col for col in df.columns if col not in train_cols+['direction']]]
    # x = df.ix[:, train_cols]
    
    # x = x.ix[:, x.columns != 'justiceName']
    
    x = pd.get_dummies(x)
    rv = cross_v(x, df)
    return rv
    # cross_v(x, df)


In [20]:
def cross_v(X, cur_df):

    cur_df['predicted'] = 0
    kf = cross_validation.KFold(len(cur_df), n_folds=5, shuffle = True)
    scores = 0
    baselines = 0
    for train, test in kf:
        score, base, pred = model(X, cur_df, train, test)
        # if scores:
        #     for i in range(len(score)):
        #         scores[i] += score[i]
        # else:
        #     scores = score
        scores += score
        baselines += base
        cur_df.loc[test,['predicted']] = pred        
        #cur_df.predicted[test] = pred
    # for score in socores:
    #     s = score/5
    #     print ('Score of CLF = ', s)
    scores = scores/5
    baselines = baselines/5
    print ('Score of CLF = ', scores)
    print ("Baseline of = ", baselines)
    rv_cols = ['predicted', 'direction', 'justiceName', 'issueArea']
    #return cur_df
    return cur_df.loc[:,rv_cols]


In [31]:

def model (X, cur_df, train, test):

    model_RF = RandomForestClassifier(n_jobs=-2, n_estimators=80, max_features=None)
    #model_KN = KNeighborsClassifier(n_jobs=-2, n_neighbors=10)
    # model_SVC = LinearSVC()
    #model_Logistic = LogisticRegression()
    modelo = model_RF
    t0 = time()
    print ('Fitting model...')

    '''
    Case for DF
    '''
    if type(X)==pd.core.frame.DataFrame:
        modelo.fit(X.ix[train, :], cur_df.direction[train].astype(int))
        y = modelo.predict(X.ix[test,:])
        score = modelo.score(X.ix[test, :], cur_df.direction[test])
        baseline = accuracy_score(cur_df.direction[test], [cur_df.direction[test].value_counts().index[0]]*len(cur_df.direction[test]))

    else:
        '''
        Case for sparse Matrix
        '''
        #train = np.array([train])
        #test = np.array([test])
        modelo.fit(X[train], cur_df.direction[train].astype(int))
        y = modelo.predict(X[test])
        score = modelo.score(X[test], cur_df.direction[test])
        baseline = accuracy_score(cur_df.direction[test], [cur_df.direction[test].value_counts().index[0]]*len(cur_df.direction[test]))

    print ('Score: ', score)
    print ('Baseline: ', baseline)
    print("Time: %0.3fs" % (time() - t0))
    print ('---------------')

    return score, baseline, y

In [356]:
cur_df = run()

Fitting model...
Score:  0.832632464256
Baseline:  0.551724137931
Time: 279.794s
---------------
Fitting model...
Score:  0.828847771236
Baseline:  0.563919259882
Time: 281.254s
---------------
Fitting model...
Score:  0.823801513877
Baseline:  0.542052144659
Time: 266.008s
---------------
Fitting model...
Score:  0.814472023559
Baseline:  0.549432057215
Time: 266.644s
---------------
Fitting model...
Score:  0.801009676062
Baseline:  0.522086663862
Time: 271.914s
---------------
Score of CLF =  0.820152689798
Baseline of =  0.54584285271


In [122]:
def acc_by_justice (df):
    rv = pd.DataFrame(index=range(0, len(judges)), columns=['justiceName', 'Predicted', 'Baseline'])
    i = 0
    for j in judges:
        #print (j)
        l = df[df['justiceName']==j]
        baseline = accuracy_score(l['direction'], [l.direction.value_counts().index[0]]*len(l.direction))
        acc = accuracy_score(l['direction'], l['predicted'])
        print (j, acc, baseline)
        rv.loc[i] = [j, acc, baseline]
        i+=1
    return rv

In [271]:
def acc_by_issue (df):
    rv = pd.DataFrame(index=range(0, len(df.issueArea.value_counts())), columns=['Issue', 'Predicted', 'Baseline'])
    i = 0
    for j in range(len(issueAreaText)):
        if j==0:
            continue
        else:
            print (issueAreaText[j])
            l = df[df['issueArea']==j]
            if len(l)!=0:
                baseline = accuracy_score(l['direction'], [l.direction.value_counts().index[0]]*len(l.direction))
                acc = accuracy_score(l['direction'], l['predicted'])
                #print (j, acc, baseline)
                rv.loc[i] = [issueAreaText[j], acc, baseline]
                i+=1
    return rv

In [357]:
byj_df = acc_by_justice(cur_df)

AScalia 0.869565217391 0.669760625305
AMKennedy 0.8405 0.587
CThomas 0.870744680851 0.689361702128
RBGinsburg 0.739967014843 0.632215503024
SGBreyer 0.750709823964 0.590573537763
JGRoberts 0.867167919799 0.583959899749
SAAlito 0.837301587302 0.637566137566
SSotomayor 0.774736842105 0.633684210526
EKagan 0.826704545455 0.613636363636


In [358]:
byi_df = acc_by_issue(df)

criminal procedure
civil rights
First Ammendment
due process
privacy
attorney or gov officials compensation
unions
economic activity
judicial power
federalism
interstate relation
federal taxation
misc
private law


In [11]:
df = run()

Fitting model...
Score:  0.773338940286
Baseline:  0.556349873844
Time: 204.727s
---------------
Fitting model...
Score:  0.759461732548
Baseline:  0.542052144659
Time: 206.173s
---------------
Fitting model...
Score:  0.751051303616
Baseline:  0.554667788057
Time: 191.055s
---------------
Fitting model...
Score:  0.75557425326
Baseline:  0.534286916281
Time: 200.519s
---------------
Fitting model...
Score:  0.777029869584
Baseline:  0.541859486748
Time: 192.169s
---------------
Score of CLF =  0.763291219859
Baseline of =  0.545843241918


In [34]:
columns = ["caseId","docketId","caseIssuesId","voteId","dateDecision","decisionType","usCite","sctCite","ledCite","lexisCite","term","naturalCourt","chief","docket","caseName","dateArgument","dateRearg","petitioner","petitionerState","respondent","respondentState","jurisdiction","adminAction","adminActionState","threeJudgeFdc","caseOrigin","caseOriginState","caseSource","caseSourceState","lcDisagreement","certReason","lcDisposition","lcDispositionDirection","declarationUncon","caseDisposition","caseDispositionUnusual","partyWinning","precedentAlteration","voteUnclear","issue","issueArea","decisionDirection","decisionDirectionDissent","authorityDecision1","authorityDecision2","lawType","lawSupp","lawMinor","majOpinWriter","majOpinAssigner","splitVote","majVotes","minVotes","justice","justiceName","vote","opinion","direction","majority","firstAgreement","secondAgreement"]

In [113]:
issueAreaText = [None, 'criminal procedure','civil rights','First Ammendment','due process',
                             'privacy','attorney or gov officials compensation','unions','economic activity',
                              'judicial power','federalism','interstate relation','federal taxation',
                              'misc','private law']

In [26]:
len(issueAreaText)

15

In [110]:
len(df.issueArea.value_counts())

12

In [24]:
df['issueArea'].value_counts()

1     2959
8     2179
2     2039
9     1634
3      937
10     743
4      426
5      280
7      252
12     216
6      174
13      49
Name: issueArea, dtype: int64

In [28]:
df.head()

,predicted,direction,justiceName,issueArea
0,1,1,AScalia,1
1,1,2,AScalia,1
2,1,2,AScalia,12
3,2,2,AScalia,3
4,2,2,AScalia,3


In [37]:
cur_df.columns[:15]

Index([                 'index',                  'issue',
                    'issueArea', 'lcDispositionDirection',
                   'petitioner',             'respondent',
                    'direction',                        0,
                              1,                        2,
                              3,                        4,
                              5,                        6,
                              7],
      dtype='object')

In [35]:
byj_df = acc_by_justice(df)

AScalia 0.813873961895 0.669760625305
AMKennedy 0.8035 0.587
CThomas 0.76170212766 0.689361702128
RBGinsburg 0.637713029137 0.632215503024
SGBreyer 0.804088586031 0.590573537763
JGRoberts 0.828320802005 0.583959899749
SAAlito 0.665343915344 0.637566137566
SSotomayor 0.682105263158 0.633684210526
EKagan 0.866477272727 0.613636363636


In [36]:
byj_df

,justiceName,Predicted,Baseline
0,AScalia,0.813874,0.669761
1,AMKennedy,0.8035,0.587
2,CThomas,0.761702,0.689362
3,RBGinsburg,0.637713,0.632216
4,SGBreyer,0.804089,0.590574
5,JGRoberts,0.828321,0.58396
6,SAAlito,0.665344,0.637566
7,SSotomayor,0.682105,0.633684
8,EKagan,0.866477,0.613636


In [124]:
byi_df = acc_by_issue(df)

criminal procedure
1 0.716120310916 0.608989523488
civil rights
2 0.753310446297 0.548798430603
First Ammendment
3 0.681963713981 0.556029882604
due process
4 0.762910798122 0.514084507042
privacy
5 0.825 0.7
attorney or gov officials compensation
6 0.827586206897 0.534482758621
unions
7 0.793650793651 0.52380952381
economic activity
8 0.821477742084 0.517668655346
judicial power
9 0.820685434517 0.546511627907
federalism
10 0.729475100942 0.566621803499
interstate relation
federal taxation
12 0.796296296296 0.689814814815
misc
13 0.714285714286 0.571428571429
private law


In [125]:
byi_df

,Issue,Predicted,Baseline
0,criminal procedure,0.71612,0.60899
1,civil rights,0.75331,0.548798
2,First Ammendment,0.681964,0.55603
3,due process,0.762911,0.514085
4,privacy,0.825,0.7
5,attorney or gov officials compensation,0.827586,0.534483
6,unions,0.793651,0.52381
7,economic activity,0.821478,0.517669
8,judicial power,0.820685,0.546512
9,federalism,0.729475,0.566622


In [53]:
g = sns.factorplot(data=byj_df, x='justiceName', y='Predicted', hue='Baseline',
                   size=6, kind="bar", palette="muted")

In [54]:
g.savefig('test3.png')

In [27]:
df.predicted.value_counts()

1    6667
2    5221
Name: predicted, dtype: int64

In [30]:
df.columns()

TypeError: 'Index' object is not callable

In [91]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib 

In [365]:
#matplotlib.rcParams.update({'font.size': 44})
fig, ax = plt.subplots()
#index = range(len(judges))
index = np.arange(len(judges))
bar_width = 0.35

opacity = 0.6
#error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, byj_df.Predicted, bar_width,
                 alpha=opacity,
                 color='r',
                 label='Predicted')
rects1 = plt.bar(index+bar_width, byj_df.Baseline, bar_width,
                 alpha=opacity,
                 color='b',
                 label='Baseline')

In [366]:
plt.xlabel('Justices', size=28, weight='bold')
plt.ylabel('Accuracy', size=28, weight='bold')
plt.title('Accuracy by Justice', size=32, weight='bold')
plt.xticks(index + bar_width, judges, rotation=17, size=24)
plt.yticks(size=24)
plt.legend(fontsize=20, loc=2)

#plt.tight_layout()
plt.show()

In [361]:
#matplotlib.rcParams.update({'font.size': 44})
fig, ax = plt.subplots()
#index = range(len(judges))
index = np.arange(len(byi_df.Issue))
bar_width = 0.35

opacity = 0.6
#error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, byi_df.Predicted, bar_width,
                 alpha=opacity,
                 color='g',
                 label='Predicted')
rects1 = plt.bar(index+bar_width, byi_df.Baseline, bar_width,
                 alpha=opacity,
                 color='y',
                 label='Baseline')

In [362]:
plt.xlabel('Issues', size=28, weight='bold')
plt.ylabel('Accuracy', size=28, weight='bold')
plt.title('Accuracy by Issues', size=32, weight='bold')
plt.xticks(index + bar_width, byi_df.Issue.tolist(), rotation=25, size=20)
plt.yticks(size=24)
plt.legend(fontsize=20, loc=2)

#plt.tight_layout()
plt.show()

In [137]:
byi_df.Issue.tolist()

['criminal procedure',
 'civil rights',
 'First Ammendment',
 'due process',
 'privacy',
 'attorney or gov officials compensation',
 'unions',
 'economic activity',
 'judicial power',
 'federalism',
 'federal taxation',
 'misc']

In [387]:
type(a)

pandas.core.series.Series

In [385]:
a =df.justiceName.value_counts()

In [389]:
years = pd.concat([byj_df, b], axis=1)

In [390]:
years

,justiceName,Predicted,Baseline,0
0,AScalia,0.869565,0.669761,2047
1,AMKennedy,0.8405,0.587,2000
2,CThomas,0.870745,0.689362,1880
3,RBGinsburg,0.739967,0.632216,1819
4,SGBreyer,0.75071,0.590574,1761
5,JGRoberts,0.867168,0.58396,798
6,SAAlito,0.837302,0.637566,756
7,SSotomayor,0.774737,0.633684,475
8,EKagan,0.826705,0.613636,352


In [178]:
years

,justiceName,Predicted,Baseline,0
0,AScalia,0.813874,0.669761,2047
1,AMKennedy,0.8035,0.587,2000
2,CThomas,0.761702,0.689362,1880
3,RBGinsburg,0.637713,0.632216,1819
4,SGBreyer,0.804089,0.590574,1761
5,JGRoberts,0.828321,0.58396,798
6,SAAlito,0.665344,0.637566,756
7,SSotomayor,0.682105,0.633684,475
8,EKagan,0.866477,0.613636,352


In [179]:
df.justiceName.value_counts()

AScalia       2047
AMKennedy     2000
CThomas       1880
RBGinsburg    1819
SGBreyer      1761
JGRoberts      798
SAAlito        756
SSotomayor     475
EKagan         352
Name: justiceName, dtype: int64

In [388]:
b =pd.DataFrame(a.values)

In [391]:
years['diff'] = years.Predicted-years.Baseline

In [394]:
years

,justiceName,Predicted,Baseline,n_votes,diff
0,AScalia,0.869565,0.669761,2047,0.199805
1,AMKennedy,0.8405,0.587,2000,0.2535
2,CThomas,0.870745,0.689362,1880,0.181383
3,RBGinsburg,0.739967,0.632216,1819,0.107752
4,SGBreyer,0.75071,0.590574,1761,0.160136
5,JGRoberts,0.867168,0.58396,798,0.283208
6,SAAlito,0.837302,0.637566,756,0.199735
7,SSotomayor,0.774737,0.633684,475,0.141053
8,EKagan,0.826705,0.613636,352,0.213068


In [400]:
years

,justiceName,Predicted,Baseline,n_votes,diff
0,AScalia,0.869565,0.669761,2047,0.199805
1,AMKennedy,0.8405,0.587,2000,0.2535
2,CThomas,0.870745,0.689362,1880,0.181383
3,RBGinsburg,0.739967,0.632216,1819,0.107752
4,SGBreyer,0.75071,0.590574,1761,0.160136
5,JGRoberts,0.867168,0.58396,798,0.283208
6,SAAlito,0.837302,0.637566,756,0.199735
7,SSotomayor,0.774737,0.633684,475,0.141053
8,EKagan,0.826705,0.613636,352,0.213068


In [393]:
years.rename(columns={0:'n_votes'}, inplace=True)

In [252]:
years['diff'] = years['diff']/100

In [186]:
sns.set(style="ticks")

In [231]:
sns.set(style="darkgrid", color_codes=True)

sns.jointplot(years['diff'], years['n_votes'], kind='reg')

TypeError: No loop matching the specified signature and casting
was found for ufunc svd_n_s

In [395]:
y, x = np.array(years['diff'], dtype=float), np.array(years['n_votes'], dtype=float)

In [396]:
import statsmodels.api as sm
results = sm.OLS(y,sm.add_constant(x)).fit()

print (results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.119
Method:                 Least Squares   F-statistic:                    0.1469
Date:                Thu, 17 Mar 2016   Prob (F-statistic):              0.713
Time:                        16:27:26   Log-Likelihood:                 14.094
No. Observations:                   9   AIC:                            -24.19
Df Residuals:                       7   BIC:                            -23.79
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.2078      0.042      4.897      0.0

/usr/local/lib/python3.4/dist-packages/scipy/stats/stats.py:1557: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [397]:
# Scatter plot
plt.scatter(x, y)

# Add correlation line
axes = plt.gca()
m, b = np.polyfit(x, y, deg=1)
X_plot = np.linspace(axes.get_xlim()[0],axes.get_xlim()[1],100)
plt.plot(X_plot, m*X_plot + b, '-')
plt.ylabel("Difference in Predicted - Baseline")
plt.xlabel("Number of Votes")
plt.title("P-Value = 71%\n Not Statistically significant")

In [398]:
plt.show()

In [254]:
kagan = acc_by_issue(df[df.justiceName=='EKagan'])

criminal procedure
1 0.833333333333 0.588888888889
civil rights
2 0.923076923077 0.730769230769
First Ammendment
3 0.666666666667 0.571428571429
due process
4 0.75 0.75
privacy
5 0.857142857143 0.714285714286
attorney or gov officials compensation
6 1.0 0.714285714286
unions
7 0.8 0.7
economic activity
8 0.9375 0.5375
judicial power
9 0.863636363636 0.613636363636
federalism
10 0.842105263158 0.684210526316
interstate relation
federal taxation
12 0.833333333333 0.666666666667
misc
13 1.0 0.75
private law


In [255]:
kagan

,Issue,Predicted,Baseline
0,criminal procedure,0.833333,0.588889
1,civil rights,0.923077,0.730769
2,First Ammendment,0.666667,0.571429
3,due process,0.75,0.75
4,privacy,0.857143,0.714286
5,attorney or gov officials compensation,1,0.714286
6,unions,0.8,0.7
7,economic activity,0.9375,0.5375
8,judicial power,0.863636,0.613636
9,federalism,0.842105,0.684211


In [269]:
def make_plots(df, name):
    fig, ax = plt.subplots()
    #index = range(len(judges))
    index = np.arange(len(df.Issue))
    bar_width = 0.35

    opacity = 0.6
    #error_config = {'ecolor': '0.3'}

    rects1 = plt.bar(index, df.Predicted, bar_width,
                     alpha=opacity,
                     color='orange',
                     label='Predicted')
    rects1 = plt.bar(index+bar_width, df.Baseline, bar_width,
                     alpha=opacity,
                     color='grey',
                     label='Baseline')
    
    plt.xlabel('Issues', size=22, weight='bold')
    plt.ylabel('Accuracy', size=22, weight='bold')
    title = 'Accuracy by Issues: '+name
    plt.title(title, size=26, weight='bold')
    plt.xticks(index + bar_width, df.Issue.tolist(), rotation=17, horizontalalignment='right', size=12)
    plt.yticks(size=24)
    plt.legend(fontsize=14, loc=2)

    #plt.tight_layout()
    plt.savefig(name+'.png', transparent=True)

In [270]:
make_plots(kagan, 'kagan')

In [370]:
def go_plots():
    v = {}
    for j in judges:
        v[j] = acc_by_issue(cur_df[cur_df.justiceName==j])
        make_plots(v[j], j)
        

In [371]:
go_plots()

criminal procedure
civil rights
First Ammendment
due process
privacy
attorney or gov officials compensation
unions
economic activity
judicial power
federalism
interstate relation
federal taxation
misc
private law
criminal procedure
civil rights
First Ammendment
due process
privacy
attorney or gov officials compensation
unions
economic activity
judicial power
federalism
interstate relation
federal taxation
misc
private law
criminal procedure
civil rights
First Ammendment
due process
privacy
attorney or gov officials compensation
unions
economic activity
judicial power
federalism
interstate relation
federal taxation
misc
private law
criminal procedure
civil rights
First Ammendment
due process
privacy
attorney or gov officials compensation
unions
economic activity
judicial power
federalism
interstate relation
federal taxation
misc
private law
criminal procedure
civil rights
First Ammendment
due process
privacy
attorney or gov officials compensation
unions
economic activity
judicial power


In [36]:
df = cur_df

In [37]:
miscl = df.direction[df.predicted != df.direction]

In [38]:
m = miscl.value_counts()

In [381]:
# The slices will be ordered and plotted counter-clockwise.
labels = 'Conservative', 'Liberal'
sizes = m
colors = ['lightcoral', 'lightskyblue']
explode = (0.1, 0)  # only "explode" the 1st slice (i.e. 'Liberal')

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.title("Misclassified")

fig = plt.figure()
#fig.patch.set_alpha(0.9)
ax = fig.gca()


In [39]:
m.sum()

2674

In [40]:
len(cur_df)

11951

In [308]:
miscl2.value_counts()

1     840
2     503
8     389
3     298
9     293
10    201
4     101
7      52
5      49
12     44
6      30
13     14
Name: issueArea, dtype: int64

In [364]:
fig.patch.set_alpha(0.1)

In [310]:
df.direction.value_counts()

1    6489
2    5399
Name: direction, dtype: int64

In [312]:
df.direction.value_counts()[1]/(df.direction.value_counts()[2]+df.direction.value_counts()[1])

0.54584454912516822

In [383]:
df.groupby('justiceName')['direction'].mean().sort(inplace=False).plot(kind='bar',color='mediumseagreen', title="Conservative-Liberal", fontsize=12)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  if __name__ == '__main__':


In [384]:
plt.show()

In [333]:
pos = df.groupby('justiceName')['direction'].mean()

In [337]:
y = y/100

In [338]:
pos

justiceName
AMKennedy     1.413000
AScalia       1.330239
CThomas       1.310638
EKagan        1.613636
JGRoberts     1.416040
RBGinsburg    1.632216
SAAlito       1.362434
SGBreyer      1.590574
SSotomayor    1.633684
Name: direction, dtype: float64

In [339]:
z = abs(1.5-pos)

In [340]:
z

justiceName
AMKennedy     0.087000
AScalia       0.169761
CThomas       0.189362
EKagan        0.113636
JGRoberts     0.083960
RBGinsburg    0.132216
SAAlito       0.137566
SGBreyer      0.090574
SSotomayor    0.133684
Name: direction, dtype: float64

In [401]:
x, y = np.array(z, dtype=float), np.array(years['diff'], dtype=float)

In [402]:
results = sm.OLS(y,sm.add_constant(x)).fit()

print (results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     1.504
Date:                Thu, 17 Mar 2016   Prob (F-statistic):              0.260
Time:                        16:31:16   Log-Likelihood:                 14.877
No. Observations:                   9   AIC:                            -25.75
Df Residuals:                       7   BIC:                            -25.36
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.1151      0.066      1.741      0.1

/usr/local/lib/python3.4/dist-packages/scipy/stats/stats.py:1557: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [405]:
# Scatter plot
plt.scatter(x, y)

# Add correlation line
axes = plt.gca()
m, b = np.polyfit(x, y, deg=1)
X_plot = np.linspace(axes.get_xlim()[0],axes.get_xlim()[1],100)
plt.plot(X_plot, m*X_plot + b, '-')
plt.ylabel("Difference in Predicted - Baseline")
plt.xlabel("Difference from 1.5")
plt.title("P-Value = 26%\n Not Statistically significant")

In [ ]:
plt.show()